In [1]:
# Imports
import datetime, json, pytz
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
from flourish_child.models import Appointment as ChildAppointment
from edc_appointment.models import Appointment as CaregiverAppointment
from edc_appointment.constants import NEW_APPT
from flourish_caregiver.models import CaregiverOffSchedule
from flourish_caregiver.helper_classes.onschedule_helper import OnScheduleHelper
from edc_visit_schedule.site_visit_schedules import site_visit_schedules

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
# Query participants who have been sequentially enrolled
sq_enrolled = Cohort.objects.values(
    'subject_identifier').annotate(cohort_count=Count('subject_identifier')).filter(cohort_count__gt=1)
child_identifiers = sq_enrolled.values_list('subject_identifier', flat=True)
child_identifiers = set(child_identifiers)

In [3]:
child_identifiers

{'B142-040990007-2-10',
 'B142-040990011-4-10',
 'B142-040990013-0-10',
 'B142-040990014-8-10',
 'B142-040990016-3-10',
 'B142-040990018-9-10',
 'B142-040990019-7-10',
 'B142-040990020-5-10',
 'B142-040990021-3-10',
 'B142-040990022-1-10',
 'B142-040990023-9-10',
 'B142-040990023-9-60',
 'B142-040990024-7-10',
 'B142-040990025-4-10',
 'B142-040990026-2-10',
 'B142-040990028-8-10',
 'B142-040990029-6-10',
 'B142-040990030-4-10',
 'B142-040990031-2-10',
 'B142-040990033-8-10',
 'B142-040990034-6-10',
 'B142-040990035-3-10',
 'B142-040990037-9-10',
 'B142-040990040-3-10',
 'B142-040990041-1-10',
 'B142-040990042-9-10',
 'B142-040990043-7-10',
 'B142-040990044-5-60',
 'B142-040990045-2-10',
 'B142-040990046-0-10',
 'B142-040990047-8-10',
 'B142-040990048-6-10',
 'B142-040990051-0-10',
 'B142-040990056-9-10',
 'B142-040990059-3-10',
 'B142-040990060-1-10',
 'B142-040990061-9-10',
 'B142-040990062-7-10',
 'B142-040990065-0-25',
 'B142-040990065-0-35',
 'B142-040990069-2-10',
 'B142-040990070

In [4]:
# Query of the sequentially enrolled, how many have multiple children
multiple_children = set()
children_tuples = {}

for idx in child_identifiers:
    latest_cosent = CaregiverChildConsent.objects.filter(
        subject_identifier=idx).latest('consent_datetime')
    children = latest_cosent.subject_consent.caregiverchildconsent_set.exclude(
        Q(subject_identifier__endswith='-35') | Q(
            subject_identifier__endswith='-46') | Q(
            subject_identifier__endswith='-56')).values_list('subject_identifier', flat=True)

    if len(set(children)) > 1:
        children = latest_cosent.subject_consent.caregiverchildconsent_set.exclude(
            subject_identifier__endswith='-35').values_list(
            'subject_identifier', flat=True)
        maternal_id = list(set(children.values_list('subject_consent__subject_identifier', flat=True)))
        multiple_children.update(set(children))
        children_tuples.update({f'{maternal_id[0]}': set(children)})

In [5]:
multiple_children = list(multiple_children)
children_tuples

{'B142-040990528-7': {'B142-040990528-7-10', 'B142-040990528-7-60'},
 'C142-040990749-9': {'C142-040990749-9-10', 'C142-040990749-9-60'},
 'B142-040990022-1': {'B142-040990022-1-10', 'B142-040990022-1-60'},
 'B142-040990099-9': {'B142-040990099-9-10', 'B142-040990099-9-60'},
 'B142-040990857-0': {'B142-040990857-0-10', 'B142-040990857-0-60'},
 'B142-040990355-5': {'B142-040990355-5-10', 'B142-040990355-5-60'},
 'B142-040990520-4': {'B142-040990520-4-10', 'B142-040990520-4-60'},
 'B142-040990172-4': {'B142-040990172-4-10', 'B142-040990172-4-60'},
 'B142-040990562-6': {'B142-040990562-6-10', 'B142-040990562-6-60'},
 'B142-040990096-5': {'B142-040990096-5-25', 'B142-040990096-5-70'},
 'B142-040990044-5': {'B142-040990044-5-10', 'B142-040990044-5-60'},
 'B142-040990023-9': {'B142-040990023-9-10', 'B142-040990023-9-60'},
 'B142-040991009-7': {'B142-040991009-7-10', 'B142-040991009-7-60'},
 'B142-040990804-2': {'B142-040990804-2-10', 'B142-040990804-2-60'},
 'B142-040990558-4': {'B142-040990

In [6]:
for child_sidx in multiple_children:
    cohorts = Cohort.objects.filter(
        subject_identifier=child_sidx).order_by('assign_datetime')
    consent = CaregiverChildConsent.objects.filter(
                subject_identifier=child_sidx).earliest('consent_datetime')
    child_count = consent.caregiver_visit_count
    maternal_sidx = consent.subject_consent.subject_identifier
    schedules = SubjectScheduleHistory.objects.filter(
        subject_identifier=maternal_sidx, ).exclude(
        schedule_name__icontains='_tb').order_by('onschedule_datetime')
    for schedule in schedules:
        onschedule_model_cls = django_apps.get_model(schedule.onschedule_model)
        onschedule_obj = onschedule_model_cls.objects.filter(
            subject_identifier=maternal_sidx,
            schedule_name=schedule.schedule_name,
            child_subject_identifier=child_sidx)
        try:
            schedule_cohort = CohortSchedules.objects.get(
                schedule_name=schedule.schedule_name).cohort_name
        except CohortSchedules.DoesNotExist:
            print('Wheet!')
        else:
            if onschedule_obj.exists() and schedule_cohort not in cohorts.values_list('name', flat=True):
                print(child_sidx, schedule_cohort, schedule.schedule_name)
                onschedule_obj.update(child_subject_identifier = '')

In [14]:
for caregiver_sidx, child_sidxs in children_tuples.items():
    for child in child_sidxs:
        child_schedules = SubjectScheduleHistory.objects.filter(
            subject_identifier=child).exclude(schedule_name__icontains='tb').order_by('onschedule_datetime')
        consent = CaregiverChildConsent.objects.filter(
            subject_identifier=child).earliest('consent_datetime')
        for schedule in child_schedules:    
            try:
                child_schedule_details = CohortSchedules.objects.get(
                    schedule_name=schedule.schedule_name)
            except CohortSchedules.DoesNotExist:
                print('Naah... can\'t be', child, caregiver_sidx)
                break
            else:
                schedule_type = child_schedule_details.schedule_type
                cohort_name = child_schedule_details.cohort_name
                child_count = consent.caregiver_visit_count

                maternal_schedule_details = CohortSchedules.objects.filter(
                    schedule_type=schedule_type,
                    cohort_name=cohort_name, child_count__isnull=False)
                if not maternal_schedule_details.exists():
                    print('Why?', child, schedule_type, cohort_name)
                else:
                    missing = []
                    found = False
                    for schedule_details in maternal_schedule_details:
                        onschedule_cls = django_apps.get_model(
                            schedule_details.onschedule_model)
                        try:
                            onschedule = onschedule_cls.objects.get(
                                subject_identifier=caregiver_sidx,
                                child_subject_identifier=child,
                                schedule_name=schedule_details.schedule_name)
                        except onschedule_cls.DoesNotExist:
                            missing.append(schedule_details.schedule_name)
                        else:
                            missing = []
                            found = True
                            break
                    if not found:
                        print(missing[0], child, caregiver_sidx)

c_sec_quart2_schedule1 B142-040990528-7-60 B142-040990528-7
c_fu_quarterly3_schedule1 C142-040990749-9-10 C142-040990749-9
c_enrol3_schedule1 B142-040990096-5-25 B142-040990096-5
b_fu3_schedule1 B142-040990096-5-25 B142-040990096-5
b_fu_quarterly3_schedule1 B142-040990096-5-25 B142-040990096-5
b_sec_quart3_schedule1 B142-040990804-2-60 B142-040990804-2


In [21]:
pid = 'B142-040990355-5'

onsch = OnScheduleCohortAFUQuarterly.objects.get(subject_identifier=pid)
visit = MaternalVisit.objects.filter(
    subject_identifier=pid, schedule_name=onsch.schedule_name).latest('report_datetime')
CaregiverOffSchedule.objects.get_or_create(
    subject_identifier=pid,
    schedule_name=onsch.schedule_name,
    defaults={'offschedule_datetime': visit.report_datetime})

try:
    onsch = OnScheduleCohortBQuarterly.objects.get(
        subject_identifier=pid, child_subject_identifier=pid + '-10')
except OnScheduleCohortBQuarterly.DoesNotExist:
    pass
else:
    onsch.delete()
    onschedule_datetime = onsch.onschedule_datetime

In [28]:
schedule_type = 'followup_quarterly'
cohort = 'cohort_b'
child_count = 1
try:
    schedule_details=CohortSchedules.objects.get(
        schedule_type=schedule_type, cohort_name=cohort, child_count=child_count)
except CohortSchedules.DoesNotExist:
    print('😳')
else:
    child_onsch = OnScheduleChildCohortAFUQuart.objects.get(subject_identifier=pid + '-10')
    helper_cls = OnScheduleHelper(subject_identifier=pid,
                                  onschedule_datetime=onsch.onschedule_datetime,
                                  cohort=cohort)
    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=schedule_details.onschedule_model, name=schedule_details.schedule_name)
    helper_cls.add_on_schedule(schedule=schedule,
                               subject_identifier=pid,
                               instance=onsch,
                               schedule_name=schedule_details.schedule_name,
                               base_appt_datetime=child_onsch.onschedule_datetime,
                               child_subject_identifier=child_onsch.subject_identifier,
                               onschedule_model_cls=OnScheduleCohortBFUQuarterly)
    appts = ChildAppointment.objects.filter(subject_identifier=child_onsch.subject_identifier,
                                            schedule_name=child_onsch.schedule_name,).exclude(appt_status=NEW_APPT)
    for appt in appts:
        try:
            caregiver_appt = CaregiverAppointment.objects.get(
                subject_identifier=pid,
                schedule_name=schedule_details.schedule_name,
                visit_code=appt.visit_code + 'M', visit_code_sequence=appt.visit_code_sequence)
        except CaregiverAppointment.DoesNotExist:
            continue
        else:
            caregiver_appt.delete()

In [29]:
pid = 'B142-040990172-4-60'
onsch = OnScheduleChildCohortAEnrollment.objects.get(subject_identifier=pid)
appt = ChildAppointment.objects.get(subject_identifier=pid, schedule_name=onsch.schedule_name)
ssh = SubjectScheduleHistory.objects.get(subject_identifier=pid, schedule_name=onsch.schedule_name)
visit = ChildVisit.objects.get(appointment=appt)
visit.delete()
appt.delete()
onsch.delete()
ssh.delete()

(1, {'edc_visit_schedule.SubjectScheduleHistory': 1})

In [13]:
onschedules = CohortSchedules.objects.filter(
    onschedule_model__startswith='flourish_caregiver').values_list('onschedule_model', flat=True)
for onschedule in onschedules:
    onschedule_cls = django_apps.get_model(onschedule)

    onschedule_objs = onschedule_cls.objects.filter(
        Q(child_subject_identifier__isnull=True) | Q(child_subject_identifier=''))

    for onschedule_obj in onschedule_objs:
        print(onschedule_obj.subject_identifier, onschedule_obj.schedule_name)
        try:
            schedule_details = CohortSchedules.objects.get(schedule_name=onschedule_obj.schedule_name)
        except CohortSchedules.DoesNotExist:
            print('No!!')
            break
        else:
            cohort = schedule_details.cohort_name
            child_idxs = CaregiverChildConsent.objects.filter(
                subject_consent__subject_identifier=onschedule_obj.subject_identifier).values_list(
                'subject_identifier', flat=True)
            child_idxs = list(set(child_idxs))
            cohort_objs = Cohort.objects.filter(subject_identifier__in=child_idxs, name=cohort)
            if cohort_objs.count() == 1:
                child_subject_identifier = cohort_objs[0].subject_identifier
                onschedule_obj.child_subject_identifier = child_subject_identifier
                onschedule_obj.save()


B142-040990037-9 c_enrol1_schedule1
B142-040990037-9 c_enrol1_schedule1
B142-040990037-9 c_enrol1_schedule1


In [2]:
pid = 'B142-040990528-7'
child_pid = 'B142-040990528-7-60'
enrol_schedule = OnScheduleCohortBSec.objects.get(
    subject_identifier=pid,
    child_subject_identifier=child_pid)
visit = MaternalVisit.objects.get(
    subject_identifier=pid, schedule_name=enrol_schedule.schedule_name)
quart_schedule = OnScheduleCohortBSecQuart.objects.get(
    subject_identifier=pid,
    child_subject_identifier=child_pid)
ssh = SubjectScheduleHistory.objects.get(subject_identifier=pid, schedule_name=quart_schedule.schedule_name)
appts = CaregiverAppointment.objects.filter(subject_identifier=pid, schedule_name=quart_schedule.schedule_name)
appts.delete()
ssh.delete()
quart_schedule.delete()

helper_cls = OnScheduleHelper(subject_identifier=pid,
                              onschedule_datetime=visit.report_datetime,
                              cohort='cohort_b_sec')
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=ssh.onschedule_model, name=quart_schedule.schedule_name)
helper_cls.add_on_schedule(schedule=schedule,
                           subject_identifier=pid,
                           instance=quart_schedule,
                           schedule_name=quart_schedule.schedule_name,
                           base_appt_datetime=visit.report_datetime,
                           child_subject_identifier=child_pid,
                           onschedule_model_cls=OnScheduleCohortBSecQuart)

In [20]:
pid = 'B142-040990096-5'
child_pid = 'B142-040990096-5-25'
child_pids = ['B142-040990096-5-25', 'B142-040990096-5-35']
for child_idx in child_pids:
    try:
        onsch = OnScheduleChildCohortCEnrollment.objects.get(subject_identifier=child_idx)
    except OnScheduleChildCohortCEnrollment.DoesNotExist:
        continue
    else:
        ssh = SubjectScheduleHistory.objects.get(
            subject_identifier=child_idx, schedule_name=onsch.schedule_name)
        appt = ChildAppointment.objects.get(subject_identifier=child_idx, schedule_name=onsch.schedule_name)
        visit = ChildVisit.objects.get(appointment=appt)
        crfs = CrfMetadata.objects.filter(
            subject_identifier=child_idx,
            schedule_name=onsch.schedule_name,
            entry_status='KEYED')
        requisitions = RequisitionMetadata.objects.filter(
            subject_identifier=child_idx,
            schedule_name=onsch.schedule_name,
            entry_status='KEYED')
        for crf in crfs:
            print(crf.model)
            model_cls = django_apps.get_model(crf.model)
            try:
                model_obj = model_cls.objects.get(child_visit=visit)
            except model_cls.DoesNotExist:
                continue
            else:
                model_obj.delete()
        for req in requisitions:
            print(req.panel_name)
            model_cls = django_apps.get_model(req.model)
            try:
                model_obj = model_cls.objects.get(child_visit=visit, panel__name=req.panel_name)
            except model_cls.DoesNotExist:
                continue
            else:
                model_obj.delete()
        visit.delete()
        appt.delete()
        ssh.delete()
        onsch.delete()

In [22]:
CaregiverChildConsent.objects.filter(subject_identifier__in=child_pids).update(cohort='cohort_b')

4

In [25]:
Cohort.objects.filter(subject_identifier__in=child_pids, name='cohort_c').delete()
cohorts = Cohort.objects.filter(subject_identifier__in=child_pids, name='cohort_b')
for cohort in cohorts:
    cohort.save()

In [26]:
# Sequentially enroll on cohort C using script. NOTE TO SELF

{'_state': <django.db.models.base.ModelState at 0x165055388>,
 'created': datetime.datetime(2023, 10, 9, 14, 49, 50, 424114, tzinfo=<UTC>),
 'modified': datetime.datetime(2023, 10, 16, 9, 42, 18, 328299, tzinfo=<UTC>),
 'user_created': 'amediphoko',
 'user_modified': 'amediphoko',
 'hostname_created': 'Ame-MacBook-Pro-M2.local',
 'hostname_modified': 'Ame-MBP-M2',
 'revision': '',
 'device_created': '99',
 'device_modified': '99',
 'id': UUID('4846972d-5675-4ba5-b1d7-56508b5ba8b0'),
 'site_id': 40,
 'subject_identifier': 'B142-040990096-5-25',
 'slug': 'b142-040990096-5-25',
 'name': 'cohort_b',
 'assign_datetime': datetime.datetime(2021, 6, 22, 13, 17, tzinfo=<UTC>),
 'enrollment_cohort': True,
 'current_cohort': True,
 'exposure_status': 'UNEXPOSED'}

In [29]:
onsch = OnScheduleCohortCEnrollment.objects.get(subject_identifier='B142-040990037-9')
ssh = SubjectScheduleHistory.objects.filter(subject_identifier='B142-040990037-9', schedule_name=onsch.schedule_name)
OnScheduleChildCohortCEnrollment.objects.filter()

<QuerySet [<SubjectScheduleHistory: SubjectScheduleHistory object (8db09a5f-e90a-4c5d-97c4-6c00deb6515a)>]>